In [1]:
import os
import csv
import json
import datetime
import requests
import geopandas
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [ ]:
# File Name for the Final COVID DataFrame
# dtfrm_file_nm = "ASN_Table.csv"

In [ ]:
# File Path for Exporting the Final Dataframe, Which Includes bts.gov, New York Times COVID, and County Polygons. 
trp_covid_geo_poly_file_pth = os.path.join("Resources", "Test_Data", "trp_covid_geo_poly_df_TEST_DATA.json")

# ========================================

# Importing Data

# ========================================

In [34]:
# New Column Names
new_date_clmn_nm = "date"
new_st_clmn_nm = "state"
new_st_pstl_code_clmn_nm = "state_postal_code"
new_cnty_clmn_nm = "county_name"
new_cnty_cnty_clmn_nm = "county_name_county"
new_cnty_fips_clmn_nm = "county_fips"
new_pop_at_hm_clmn_nm = "pop_at_home"
new_pop_not_at_hm_clmn_nm = "pop_not_at_home"
new_nmbr_trps_clmn_nm = "trips"
new_cases_clmn_nm = "covid_cases"
new_dths_clmn_nm = "covid_deaths"

# ---------------------------------------------------------------------

## Import bts.gov Trip Data

# ---------------------------------------------------------------------

In [3]:
bts_gov_trps_csv_new_hdr_nm = [new_date_clmn_nm, new_st_pstl_code_clmn_nm, new_cnty_clmn_nm, new_pop_at_hm_clmn_nm, new_pop_not_at_hm_clmn_nm, new_nmbr_trps_clmn_nm]

In [ ]:
# File Path for Importing the bts.gov Trips Data .csv File
bts_gov_trps_file_pth = os.path.join("Resources", "bts.gov", "Trips_by_Distance.csv")

# Convert the CSV to a Dataframe
bts_gov_trps_csv_header = pd.read_csv(bts_gov_trps_file_pth, names= ["DateTest", "State Postal Code", "County Name", "Population Staying at Home", "Population Not Staying at Home", "Number of Trips"])

In [ ]:
bts_gov_trps_csv_header.head()

In [6]:
# File Path for Importing the bts.gov Trips Data .csv File
bts_gov_trps_file_pth = os.path.join("Resources", "bts.gov", "Trips_by_Distance.csv")

# Convert the CSV to a Dataframe
bts_gov_trps_df = pd.read_csv(bts_gov_trps_file_pth, usecols= ["Date", "State FIPS", "State Postal Code", "County FIPS", "County Name", "Population Staying at Home", "Population Not Staying at Home", "Number of Trips"])

In [7]:
bts_gov_trps_df.head()

,Date,State FIPS,State Postal Code,County FIPS,County Name,Population Staying at Home,Population Not Staying at Home,Number of Trips
0,2019/01/01,29.0,MO,29171.0,Putnam County,"1,155","3,587","12,429"
1,2019/01/01,2.0,AK,2164.0,Lake and Peninsula Borough,NaN,NaN,NaN
2,2019/01/01,1.0,AL,1001.0,Autauga County,"9,624","45,807","132,004"
3,2019/01/01,1.0,AL,1003.0,Baldwin County,"44,415","172,941","534,520"
4,2019/01/01,1.0,AL,1005.0,Barbour County,"4,782","20,023","67,658"


In [ ]:
print(bts_gov_trps_df.count())

In [ ]:
bts_gov_trps_df.dtypes

In [ ]:
bts_gov_trps_df.keys()

In [15]:
# Change the Column Names
bts_gov_trps_df_test = bts_gov_trps_df.rename(columns = {"Date": new_date_clmn_nm, "State Postal Code": new_st_pstl_code_clmn_nm, "County FIPS": new_cnty_fips_clmn_nm, "County Name": new_cnty_clmn_nm, "Population Staying at Home": new_pop_at_hm_clmn_nm, "Population Not Staying at Home": new_pop_not_at_hm_clmn_nm,"Number of Trips": new_nmbr_trps_clmn_nm})
bts_gov_trps_df_test.keys()

Index(['date', 'State FIPS', 'state_postal_code', 'county_fips', 'county',
       'pop_at_home', 'pop_not_at_home', 'trips'],
      dtype='object')

# ---------------------------------------------------------------------

## Import New York Times COVID-19 Data

# ---------------------------------------------------------------------

In [8]:
# File Path for Importing the New York Times COVID-19 Data .csv File
nytimes_covid_19_file_pth = os.path.join("Resources", "github.com_nytimes_covid_19_data", "us-counties20200121_to_20201130.csv")

# Convert the CSV to a Dataframe
nytimes_covid_19_df = pd.read_csv(nytimes_covid_19_file_pth)

In [ ]:
nytimes_covid_19_df

In [10]:
nytimes_covid_19_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [ ]:
nytimes_covid_19_df.keys()

In [16]:
# Change the Column Names
nytimes_covid_19_df_test = nytimes_covid_19_df.rename(columns = {"date": new_date_clmn_nm, "state": new_st_clmn_nm, "fips": new_cnty_fips_clmn_nm, "county": new_cnty_clmn_nm, "cases": new_cases_clmn_nm, "deaths": new_dths_clmn_nm})
nytimes_covid_19_df_test.keys()
# nytimes_covid_19_df_test = nytimes_covid_19_df_test[["date_covid", "state_covid", "cases_covid","deaths_covid"]]

Index(['date', 'county', 'state', 'county_fips', 'covid_cases',
       'covid_deaths'],
      dtype='object')

# ---------------------------------------------------------------------

# ******NOT WORKING******

# ---------------------------------------------------------------------

In [ ]:
# How to save only selected columns of a DataFrame to HDF5
# https://stackoverflow.com/questions/27878780/pandas-how-to-save-only-selected-columns-of-a-dataframe-to-hdf5
cols_to_keep = [['date', 'state', 'cases', 'deaths']]
nytimes_covid_19_df.loc[:, cols_to_keep].to_hdf(...)

In [ ]:
nytimes_covid_19_df = nytimes_covid_19_df[['date', 'state', 'cases', 'deaths']]
# nytimes_covid_19_df = nytimes_covid_19_df["date", "state", "cases", "deaths"]
nytimes_covid_19_df

# ---------------------------------------------------------------------

In [ ]:
# Shape of the New York Times COVID-19 Dataframe
# len(nytimes_covid_19_df)
nytimes_covid_19_df.shape

In [ ]:
# Number of Rows that do Contain Null Values
print(nytimes_covid_19_df.count())

In [ ]:
# Data Types in the New York Dataframe
nytimes_covid_19_df.dtypes

# ---------------------------------------------------------------------

# Import US State Name and Postal Code Data
# From the U.S. Bureau of Labor Statistics (bls.gov)

# ---------------------------------------------------------------------

In [ ]:
bls_gov_st_st_code_df = pd.read_html("https://www.infoplease.com/us/postal-information/state-abbreviations-and-state-postal-codes")
len(bls_gov_st_st_code_df)

In [ ]:
bls_gov_st_st_code_df = bls_gov_st_st_code_df[0]
bls_gov_st_st_code_df

# ---------------------------------------------------------------------

# Import County Polygons .json File

# ---------------------------------------------------------------------

In [ ]:
# # File Path for Importing the County Geometry Polygons.json File
# cnty_geo_file_pth = os.path.join("Resources", "County_Geometry", "county_geometry.json")

# # File Path for Importing the Example Househole Income Geometry Polygons.json File
# household_income_geo_file_pth = os.path.join("Resources", "County_Geometry", "Median_Household_Income_2016.geojson")

# File Path for Import the Example Househole Income Geometry Polygons.json File
us_cnty_bndrs_file_pth = os.path.join("Resources", "County_Geometry", "us-county-boundaries.geojson")

# Convert the CSV to a Dataframe
us_cnty_bndrs_df = geopandas.read_file(us_cnty_bndrs_file_pth, usecols= ["stusab", "namelsad", "state_name", "geometry"])
us_cnty_bndrs_df.head()

In [ ]:
us_cnty_bndrs_df.keys()

In [ ]:
# Change the Column Names
us_cnty_bndrs_df_test = us_cnty_bndrs_df.rename(columns = {"stusab": new_st_pstl_code_clmn_nm, "namelsad": new_cnty_clmn_nm,"state_name": new_st_clmn_nm})
us_cnty_bndrs_df_test.keys()
# nytimes_covid_19_df_test = nytimes_covid_19_df_test[["date_covid", "state_covid", "cases_covid","deaths_covid"]]

In [ ]:
us_cnty_bndrs_df_test = us_cnty_bndrs_df_test[[new_st_clmn_nm, new_st_pstl_code_clmn_nm, new_cnty_clmn_nm, "geometry"]]
us_cnty_bndrs_df_test.head()

In [ ]:
household_income_covid_19_df["Cnty_Name"].value_counts()

In [ ]:
1) Only import the columns I need. 
2) Change the data types, some of the data types use less memory
    2.1) you know that a column should only have positive integers, use unsigned integer type 
        (uint32) instead of the regular int type (or worse — float, which may sometimes happen 
        automatically).
3) 

# ---------------------------------------------------------------------

# Import County Points
# From the U.S. Bureau of Labor Statistics (bls.gov)

# ---------------------------------------------------------------------

In [ ]:
# # File Path for Importing the County Geometry Polygons.json File
# cnty_geo_file_pth = os.path.join("Resources", "County_Geometry", "county_geometry.json")

# # File Path for Importing the Example Househole Income Geometry Polygons.json File
# household_income_geo_file_pth = os.path.join("Resources", "County_Geometry", "Median_Household_Income_2016.geojson")

# File Path for Import the Example Househole Income Geometry Polygons.json File
us_cnty_pnts_file_pth = os.path.join("Resources", "County_Points", "county_geometry.json")

us_cnty_pnts_df = pd.read_json(us_cnty_pnts_file_pth)


# # Convert the CSV to a Dataframe
# us_cnty_bndrs_df = geopandas.read_file(us_cnty_bndrs_file_pth)
# us_cnty_bndrs_df.head()

In [ ]:
bls_gov_st_st_code_df = pd.read_html("https://www.infoplease.com/us/postal-information/state-abbreviations-and-state-postal-codes")
len(bls_gov_st_st_code_df)

In [ ]:
bls_gov_st_st_code_df = bls_gov_st_st_code_df[0]
bls_gov_st_st_code_df

# ---------------------------------------------------------------------

## Import simplemaps_uscities Data

# ---------------------------------------------------------------------

In [12]:
# File Path for Importing the New York Times COVID-19 Data .csv File
simplemaps_uscities_file_pth = os.path.join("Resources", "simplemaps_uscities", "uscities.csv")

# Convert the CSV to a Dataframe
simplemaps_uscities_df = pd.read_csv(simplemaps_uscities_file_pth, usecols= ["county_fips", "county_name", "lat", "lng"])

In [13]:
simplemaps_uscities_df.head()

,county_fips,county_name,lat,lng
0,36061,New York,40.6943,-73.9249
1,6037,Los Angeles,34.1139,-118.4068
2,17031,Cook,41.8373,-87.6862
3,12086,Miami-Dade,25.7839,-80.2102
4,48113,Dallas,32.7936,-96.7662


# ---------------------------------------------------------------------

## Import Texas County Centers Data

# ---------------------------------------------------------------------

In [47]:
# File Path for Importing the bts.gov Trips Data .csv File
tx_cnty_cntrs_file_pth = os.path.join("Resources", "County_Points", "texas_county_centers.csv")

# Convert the CSV to a Dataframe
tx_cnty_cntrs_df = pd.read_csv(tx_cnty_cntrs_file_pth, usecols= ["CNAME","XCoord", "YCoord"])
tx_cnty_cntrs_df.head()

,XCoord,YCoord,CNAME
0,-95.652315,31.813202,ANDERSON
1,-102.637310,32.304938,ANDREWS
2,-94.611708,31.254821,ANGELINA
3,-96.998991,28.106930,ARANSAS
4,-98.687025,33.615379,ARCHER


# ========================================

# Null Values

# ========================================

# ---------------------------------------------------------------------

## bts.gov Trip Data Null Values

# ---------------------------------------------------------------------

In [ ]:
# Returns true if the column has null values
bts_gov_trps_df.isnull().any()
# OR
# bts_gov_trps_df.isnull().any(axis=0) #(axis=0 is the defult)

In [ ]:
# Rows with Null Values in the Trips Dataframe 
bts_gov_trps_df.isnull().sum()

In [ ]:
# Returns a Dataframe Containing Only the Rows With Null Values
bts_gov_trps_df_null_val = bts_gov_trps_df[bts_gov_trps_df.isnull().any(axis=1)]

# Returns the Number of Cells Within a Column With Null Values
print(bts_gov_trps_df_null_val.count())
bts_gov_trps_df_null_val

# ---------------------------------------------------------------------

### Dateframe with Totals for each State and the Nation Per Day

# ---------------------------------------------------------------------

In [ ]:
# Array of the Columns Where the Null Values will be Converted to 0
# These are the Columns that Should Have a Zero Values Instead of a Null Values
clmns_to_cnvrt_null_vls_to_0 =["Population Staying at Home", "Population Not Staying at Home","Number of Trips","Number of Trips <1", "Number of Trips 1-3", "Number of Trips 1-3", "Number of Trips 3-5", "Number of Trips 5-10", "Number of Trips 10-25", "Number of Trips 25-50", "Number of Trips 50-100", "Number of Trips 100-250", "Number of Trips 250-500", "Number of Trips >=500"]

In [ ]:
# Drop the Rows that With Null Values (NaN) in ALL the Columns in the ["clmns_to_cnvrt_null_vls_to_0"] array: if All the Values are Null.
bts_gov_trps_df_null_val_lvl_st_ntnl = bts_gov_trps_df_null_val.dropna(how="all", subset=clmns_to_cnvrt_null_vls_to_0)
print(bts_gov_trps_df_null_val_lvl_st_ntnl.count())
bts_gov_trps_df_null_val_lvl_st_ntnl

#### The Above Dataframe Looks Like the Totals for Each State and the Nation Per Day.

# ---------------------------------------------------------------------

### Dataframe With Each State's Total Trips Per Day

# ---------------------------------------------------------------------

In [ ]:
# Dataframe With the Total State Trips Per Day
bts_gov_trps_df_cnty_clmn_null_lvl_st = bts_gov_trps_df_null_val_lvl_st_ntnl.dropna(how="all", subset=["State Postal Code"])
print(bts_gov_trps_df_cnty_clmn_null_lvl_st.count())
bts_gov_trps_df_cnty_clmn_null_lvl_st

In [ ]:
# Which Columns Have Null Values in the State Level Dataframe
bts_gov_trps_df_cnty_clmn_null_lvl_st.isnull().any()

#### Correct, Only the County Columns have Null Values. Since This is a Daily State Totals Dataframe. 

# ---------------------------------------------------------------------

### Dataframe With Total National Trips Per Day

# ---------------------------------------------------------------------

In [ ]:
# Dataframe the Total National Trips Per Day
bts_gov_trps_df_NULL_VALUES_lvl_ntnl = bts_gov_trps_df_NULL_VALUES_lvl_st_ntnl[bts_gov_trps_df_NULL_VALUES_lvl_st_ntnl["Level"] == "National"]
print(bts_gov_trps_df_NULL_VALUES_lvl_ntnl.count())
bts_gov_trps_df_NULL_VALUES_lvl_ntnl

In [ ]:
# Which Columns Have Null Values in the National Level Dataframe
bts_gov_trps_df_NULL_VALUES_lvl_ntnl.isnull().any()

#### Correct, the State and County Columns have Null Values. Since This is a Daily National Totals Dataframe. 

In [ ]:
bts_gov_trps_df_NULL_VALUES_lvl_ntnl = bts_gov_trps_df_NULL_VALUES_lvl_ntnl

# ---------------------------------------------------------------------

### Dataframe With Total County Trips Per Day

# ---------------------------------------------------------------------

In [ ]:
# Dataframe the Total County Trips Per Day
bts_gov_trps_df_lvl_cnty = bts_gov_trps_df[bts_gov_trps_df["Level"] == "County"]
print(bts_gov_trps_df_lvl_cnty.count())
bts_gov_trps_df_lvl_cnty

In [ ]:
# Which Columns Have Null Values in the County Level Dataframe
bts_gov_trps_df_lvl_cnty.isnull().any()

#### #### I am Not Sure Why the Population and Trips Columns Have Null Values. I am Assuming They are Null Because No Trips Were Taken?

In [ ]:
bts_gov_trps_df_NULL_VALUES_null_dltd["Level"].unique()

# ---------------------------------------------------------------------

#### Convert All the Null Values in the "clmns_to_cnvrt_null_vls_to_0" Array to Zero

# ---------------------------------------------------------------------

In [ ]:
bts_gov_trps_df_lvl_cnty[clmns_to_cnvrt_null_vls_to_0] = bts_gov_trps_df_lvl_cnty[clmns_to_cnvrt_null_vls_to_0].replace(np.nan, 0)
bts_gov_trps_df_lvl_cnty

In [ ]:
# Which Columns Have Null Values in the County Level Dataframe
bts_gov_trps_df_lvl_cnty.isnull().any()

#### Correct, Now all the Columns Have a Value.

# ---------------------------------------------------------------------

## New York Times COVID-19 Data Null Values

# ---------------------------------------------------------------------

In [ ]:
# Columns with Null Values in the New York Times Dataframe 
nytimes_covid_19_df.isnull().any()

In [ ]:
# Rows with Null Values in the New York Times Dataframe 
nytimes_covid_19_df.isnull().sum()

In [ ]:
# Returns a Dataframe Containing Only the Rows With Null Values
nytimes_covid_19_df_null_val = nytimes_covid_19_df[nytimes_covid_19_df.isnull().any(axis=1)]

# Returns the Number of Cells Within a Column With Null Values
print(nytimes_covid_19_df_null_val.count())
nytimes_covid_19_df_null_val

#### Since the ["fips"] Column has a Lot of Null Values, Lets Isolate the ["deaths"] Column to See How many Null Values are  in the ["deaths"] Column.

In [ ]:
# Isolated the Null Values in the ["deaths"] Column.
nytimes_covid_19_df_dth_clmn_null_val = nytimes_covid_19_df[nytimes_covid_19_df["deaths"].isnull()]
print(len(nytimes_covid_19_df_dth_clmn_null_val))
print(nytimes_covid_19_df_dth_clmn_null_val["state"].value_counts())
nytimes_covid_19_df_dth_clmn_null_val

# *******Puerto Rico has no reported deaths?*******

# ---------------------------------------------------------------------

#### Convert All the Null Values in the ["deaths"] Column to Zero

# ---------------------------------------------------------------------

In [ ]:
# Replace the Null Values With 0
nytimes_covid_19_df_no_null_val_dths = nytimes_covid_19_df
nytimes_covid_19_df_no_null_val_dths["deaths"] = nytimes_covid_19_df_no_null_val_dths["deaths"].replace(np.nan, 0)
nytimes_covid_19_df_no_null_val_dths

In [ ]:
# Rows with Null Values
nytimes_covid_19_df_no_null_val_dths[nytimes_covid_19_df_no_null_val_dths["deaths"].isnull()]

# ========================================

# Convert the Date Column to a datetime Date Type & Convert the Other Columns with Numbers into an int64 Data Type

# ========================================

# ---------------------------------------------------------------------

## bts.gov Trip Data

# ---------------------------------------------------------------------

# ---------------------------------------------------------------------

### The State Level Dataframe

# ---------------------------------------------------------------------

# ---------------------------------------------------------------------

#### Remove the Commas in the [clmns_to_cnvrt_null_vls_to_0] Columns Array in the State Level Dataframe

# ---------------------------------------------------------------------

In [ ]:
# Remove the Commas from the [clmns_to_cnvrt_null_vls_to_0] Array.
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms = bts_gov_trps_df_cnty_clmn_null_lvl_st
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms[clmns_to_cnvrt_null_vls_to_0] = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms[clmns_to_cnvrt_null_vls_to_0].replace(',','', regex=True)
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms

In [ ]:
# View Data Types
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms.dtypes

In [ ]:
# Change the Data Type of the Columns in the [clmns_to_cnvrt_null_vls_to_0] Array from a String to a int64
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64 = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64[clmns_to_cnvrt_null_vls_to_0] = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64[clmns_to_cnvrt_null_vls_to_0].astype("int64")

# ---------------------------------------------------------------------

#### Convert the ["Date"] Column to a datetime64[ns] Data Type in the State Level Dataframe

# ---------------------------------------------------------------------

In [ ]:
# Convert the ["Date"] Column from a String Data Type to a datetime64[ns] Data Type
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm["Date"]  = pd.to_datetime(bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm["Date"])

print(bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm["Date"].dtypes)
print("-------------------------")
print(bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm["Date"])

In [ ]:
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm.dtypes

# ---------------------------------------------------------------------

## New York Times COVID-19 Data

# ---------------------------------------------------------------------

# ---------------------------------------------------------------------

### Change Deaths From a float to an int Data Type in the COVID-19 Dataframe

# ---------------------------------------------------------------------

# ---------------------------------------------------------------------

#### Check the Values After the Decimal to Determine if They are Greater Than 0.

# ---------------------------------------------------------------------

In [ ]:
# Check the Numbers After a Decimal Point
# https://stackoverflow.com/questions/57900886/how-to-check-numbers-after-decimal-point

nytimes_covid_19_df_no_null_val_dths_no_dcml_dths = nytimes_covid_19_df_no_null_val_dths
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths['check_point'] = np.modf(nytimes_covid_19_df_no_null_val_dths_no_dcml_dths['deaths'])[0].ne(0).astype(int)
# nytimes_covid_19_df_no_null_val_dths_dcml_chck = nytimes_covid_19_df_no_null_val_dths_dcml_chck['check_point']
nytimes_covid_19_df_no_null_val_dths_dcml_chck_dcml_not_zero = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths[nytimes_covid_19_df_no_null_val_dths_no_dcml_dths["check_point"]!=0]
nytimes_covid_19_df_no_null_val_dths_dcml_chck_dcml_not_zero
# nytimes_covid_19_df_no_null_val_dths_dcml_chck["state"].value_counts()

# Before Rounding, Need to Convert the ["deaths] Column Datatype From a String to a int

In [ ]:
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths.dtypes

In [ ]:
# Convert to ["deaths"] Column to an int Data Type
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths["deaths"] = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths["deaths"].astype("int64")
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths.dtypes

In [ ]:
# Round the Death Rate Up to a Whole Number.
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths
# nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_rnd_ceil["deaths"] = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_rnd_ceil["deaths"].apply(np.ceil)

nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths.round({"deaths": 0})
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths

# ---------------------------------------------------------------------

### Convert the ["Date"] Column to a datetime64[ns] Data Type in the State Level Dataframe

# ---------------------------------------------------------------------

In [ ]:
# Convert the ["Date"] Column from a String Data Type to a datetime64[ns] Data Type
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date["date"]  = pd.to_datetime(nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date["date"])

print(nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date["date"].dtypes)
print("-------------------------")
print(nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date["date"])

In [ ]:
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date.dtypes

In [ ]:
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date

# ========================================

# Dataframes with Limited Columns

# ========================================

# ---------------------------------------------------------------------

## State Dataframe with Limited Columns

# ---------------------------------------------------------------------

In [ ]:
# New State Dataframe With the Following Columns: ["Date", "State Postal Code", "Population Staying at Home", "Population Not Staying at Home","Number of Trips"] 
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn[["Date", "State Postal Code", "Population Staying at Home", "Population Not Staying at Home","Number of Trips"]]
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn

In [ ]:
# Check the Data Types of the Dataframe
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn.dtypes

In [ ]:
# Check the Dataframe for Null Values
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn.isnull().any()

In [ ]:
# Number of Unique Values in the Date Column of the State Dataframe
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn["Date"].value_counts()

In [ ]:
# Number of Non-NA cells for each column in the Date Column of the State Dataframe
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn.count()

In [ ]:
# Total Number of Rows in the Dataframe
len(bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn)

In [ ]:
# The Shape of the Dataframe
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn.shape

# ---------------------------------------------------------------------

## COVID-19 Dataframe with Limited Columns

# ********NOT WORKING********

# ---------------------------------------------------------------------

In [ ]:
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date.

In [ ]:
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date_lmtd_clmn = nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date["date", "cases"]
nytimes_covid_19_df_no_null_val_dths_no_dcml_dths_int_dths_rnd_dths_dttm_date_lmtd_clmn

# ---------------------------------------------------------------------

## Texas County Polygons Dataframe with Limited Columns

# ---------------------------------------------------------------------

In [ ]:
# Column Names in the US County Boundary Dataframe
us_cnty_bndrs_df.keys()

In [ ]:
us_cnty_bndrs_df_lmtd_clmns = us_cnty_bndrs_df
us_cnty_bndrs_df_lmtd_clmns = us_cnty_bndrs_df_lmtd_clmns[["countyfp_nozero", "countyfp", "namelsad", "statefp", "state_name", "geometry"]]
us_cnty_bndrs_df_lmtd_clmns["geometry"]

# ========================================

# Change the Name of the Columns

# ========================================

# ---------------------------------------------------------------------

## Change the Column Names in the State Trips Dataframe

# ---------------------------------------------------------------------

In [ ]:
# New Column Names
new_date_clmn_nm = "date"
new_st_clmn_nm = "state"
new_st_pstl_code_clmn_nm = "state_postal_code"
new_pop_at_hm_clmn_nm = "pop_at_home"
new_pop_not_at_hm_clmn_nm = "pop_not_at_home"
new_nmbr_trps_clmn_nm = "number_trips"
new_cases_clmn_nm = "covid_cases"
new_dths_clmn_nm = "covid_deaths"

In [ ]:
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn.keys()

In [ ]:
# Changed the Names of the Column Headers
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn_clmn_nm_chng = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn_clmn_nm_chng = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn_clmn_nm_chng ({"Date": new_date_clmn_nm, "State Postal Code": new_st_pstl_code_clmn_nm, "Population Staying at Home": new_pop_at_hm_clmn_nm, "Population Not Staying at Home": new_pop_not_at_hm_clmn_nm, "Number of Trips": new_nmbr_trps_clmn_nm}, axis = 1)
# bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn_clmn_nm_chng#.head()

# ========================================

# Add the State Name & State Postal Code

# ========================================

# ---------------------------------------------------------------------

## Add the State Postal Code in the Trips Dataframe

# ---------------------------------------------------------------------

In [ ]:

bls_gov_st_st_code_df.keys()

In [ ]:
bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn.keys()

In [ ]:
# Replace all the Abbreviations with the Full State Name 
for t in range(len(State_Abbreviations_Code_Table_df)):

    for i in range(len(avg_auto_insnc_concat_df_clnd)):

        if State_Abbreviations_Code_Table_df.iloc[t]["Code"] == avg_auto_insnc_concat_df_clnd.iloc[i]["State"]:
            avg_auto_insnc_concat_df_clnd.iloc[i]["State"] = State_Abbreviations_Code_Table_df.iloc[t]["Description"]

In [ ]:
# ********NOT WORKING********

# ========================================

# Change the Name of the Dataframes

# ========================================

# ---------------------------------------------------------------------

## Change the State Trips Dataframe

# ---------------------------------------------------------------------

In [ ]:
bts_gov_trps_df_FINAL = bts_gov_trps_df_cnty_clmn_null_lvl_st_no_cmms_int64_dttm_lmtd_clmn
bts_gov_trps_df_FINAL

# ========================================

# Join the Dataframes

# ========================================

# ---------------------------------------------------------------------

## Join the State Trips Dataframe with the COVID-19 Dataframe

# ---------------------------------------------------------------------

In [17]:
bts_gov_trps_df_test.head()
print(bts_gov_trps_df_test.shape)

(2207054, 8)


In [18]:
bts_gov_trps_df_test_2 = bts_gov_trps_df_test.dropna()
bts_gov_trps_df_test_2.shape

(2148011, 8)

In [19]:
# Filter Data on ["State Postal Code"] Column for "TX"
bts_gov_trps_df_test_2 = bts_gov_trps_df_test_2[bts_gov_trps_df_test_2[new_st_pstl_code_clmn_nm] == "TX"]
bts_gov_trps_df_test_2.shape
# bts_gov_trps_df_test_2

(172711, 8)

In [ ]:
# bts_gov_trps_df_test_2_grpby = bts_gov_trps_df_test_2.groupby(["county_fips"], as_index = False)

In [ ]:
# bts_gov_trps_df_test_2_grpby.head()

In [20]:
bts_gov_trps_df_test_2["date"]  = pd.to_datetime(bts_gov_trps_df_test_2["date"])
bts_gov_trps_df_test_2.dtypes

date                 datetime64[ns]
State FIPS                  float64
state_postal_code            object
county_fips                 float64
county                       object
pop_at_home                  object
pop_not_at_home              object
trips                        object
dtype: object

In [21]:
# Filter Data on ["State Postal Code"] Column for "AK"
nytimes_covid_19_df_test = nytimes_covid_19_df_test[nytimes_covid_19_df_test["state"] == "Texas"]
nytimes_covid_19_df_test.shape

(59128, 6)

In [ ]:
nytimes_covid_19_df_test

In [22]:
nytimes_covid_19_df_test["county"].value_counts()

Bexar         293
Fort Bend     272
Harris        271
Collin        267
Montgomery    266
             ... 
Sterling      110
Borden         69
King           49
Loving         14
Unknown         1
Name: county, Length: 255, dtype: int64

In [ ]:
# nytimes_covid_19_df_test_grpby = nytimes_covid_19_df_test.groupby(["county_fips"], as_index = False)

In [ ]:
# nytimes_covid_19_df_test_grpby.head()

In [23]:
nytimes_covid_19_df_test["date"]  = pd.to_datetime(nytimes_covid_19_df_test["date"])
nytimes_covid_19_df_test.dtypes

date            datetime64[ns]
county                  object
state                   object
county_fips            float64
covid_cases              int64
covid_deaths           float64
dtype: object

In [24]:
# new_cnty_fips_clmn_nm
trp_covid_df = pd.merge(bts_gov_trps_df_test_2, nytimes_covid_19_df_test, on = ["date", "county_fips"], suffixes=("_trps", "_covid"), how = "inner")

In [25]:
trp_covid_df.shape

(56735, 12)

In [26]:
trp_covid_df.dropna(how = "any")

,date,State FIPS,state_postal_code,county_fips,county_trps,pop_at_home,pop_not_at_home,trips,county_covid,state,covid_cases,covid_deaths
0,2020-02-12,48.0,TX,48029.0,Bexar County,"326,140","1,659,909","7,145,685",Bexar,Texas,1,0.0
1,2020-02-13,48.0,TX,48029.0,Bexar County,"317,245","1,668,804","7,326,573",Bexar,Texas,2,0.0
2,2020-02-14,48.0,TX,48029.0,Bexar County,"314,930","1,671,119","7,859,381",Bexar,Texas,2,0.0
3,2020-02-15,48.0,TX,48029.0,Bexar County,"355,894","1,630,155","7,104,962",Bexar,Texas,2,0.0
4,2020-02-16,48.0,TX,48029.0,Bexar County,"399,704","1,586,345","6,281,204",Bexar,Texas,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
56730,2020-11-21,48.0,TX,48031.0,Blanco County,"2,481","9,221","36,299",Blanco,Texas,178,6.0
56731,2020-11-21,48.0,TX,48033.0,Borden County,134,514,"2,987",Borden,Texas,5,0.0
56732,2020-11-21,48.0,TX,48035.0,Bosque County,"4,714","13,977","50,067",Bosque,Texas,507,15.0
56733,2020-11-21,48.0,TX,48037.0,Bowie County,"23,241","71,083","257,762",Bowie,Texas,2479,97.0


In [39]:
trp_covid_df_filter = trp_covid_df[(trp_covid_df['date']>'2020-11-01') & (trp_covid_df['date']<'2020-11-30')]

In [40]:
trp_covid_df_filter["county_trps"].value_counts()

Matagorda County    20
Ochiltree County    20
Knox County         20
Blanco County       20
Milam County        20
                    ..
Kaufman County      20
Hidalgo County      20
Zapata County       20
Callahan County     20
Loving County        5
Name: county_trps, Length: 254, dtype: int64

In [41]:
trp_covid_df_filter

,date,State FIPS,state_postal_code,county_fips,county_trps,pop_at_home,pop_not_at_home,trips,county_covid,state,covid_cases,covid_deaths
51670,2020-11-02,48.0,TX,48495.0,Winkler County,"1,994","5,726","23,034",Winkler,Texas,153,5.0
51671,2020-11-02,48.0,TX,48001.0,Anderson County,"10,765","47,292","174,643",Anderson,Texas,2979,39.0
51672,2020-11-02,48.0,TX,48003.0,Andrews County,"4,014","14,114","54,811",Andrews,Texas,667,10.0
51673,2020-11-02,48.0,TX,48005.0,Angelina County,"18,309","68,783","251,533",Angelina,Texas,2922,93.0
51674,2020-11-02,48.0,TX,48007.0,Aransas County,"6,740","17,052","52,929",Aransas,Texas,351,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...
56730,2020-11-21,48.0,TX,48031.0,Blanco County,"2,481","9,221","36,299",Blanco,Texas,178,6.0
56731,2020-11-21,48.0,TX,48033.0,Borden County,134,514,"2,987",Borden,Texas,5,0.0
56732,2020-11-21,48.0,TX,48035.0,Bosque County,"4,714","13,977","50,067",Bosque,Texas,507,15.0
56733,2020-11-21,48.0,TX,48037.0,Bowie County,"23,241","71,083","257,762",Bowie,Texas,2479,97.0


# ---------------------------------------------------------------------

## Join the Trips & COVID-19 Dataframe with the Polygon Dataframe

# ---------------------------------------------------------------------

In [ ]:
# # Filter Data on ["State Postal Code"] Column for "AK"
# us_cnty_bndrs_df_test = us_cnty_bndrs_df_test[us_cnty_bndrs_df_test["state"] == "Texas"]
# us_cnty_bndrs_df_test.shape

In [38]:
# us_cnty_bndrs_df_test.head()

In [42]:
# Change the Column Names
# new_cnty_clmn_nm = "county_name"
# new_cnty_cnty_clmn_nm = "county_name_county"
trp_covid_df_filter = trp_covid_df_filter.rename(columns = {"county_trps": new_cnty_cnty_clmn_nm, "county_covid": new_cnty_clmn_nm})
trp_covid_df_filter
# nytimes_covid_19_df_test = nytimes_covid_19_df_test[["date_covid", "state_covid", "cases_covid","deaths_covid"]]

,date,State FIPS,state_postal_code,county_fips,county_name_county,pop_at_home,pop_not_at_home,trips,county_name,state,covid_cases,covid_deaths
51670,2020-11-02,48.0,TX,48495.0,Winkler County,"1,994","5,726","23,034",Winkler,Texas,153,5.0
51671,2020-11-02,48.0,TX,48001.0,Anderson County,"10,765","47,292","174,643",Anderson,Texas,2979,39.0
51672,2020-11-02,48.0,TX,48003.0,Andrews County,"4,014","14,114","54,811",Andrews,Texas,667,10.0
51673,2020-11-02,48.0,TX,48005.0,Angelina County,"18,309","68,783","251,533",Angelina,Texas,2922,93.0
51674,2020-11-02,48.0,TX,48007.0,Aransas County,"6,740","17,052","52,929",Aransas,Texas,351,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...
56730,2020-11-21,48.0,TX,48031.0,Blanco County,"2,481","9,221","36,299",Blanco,Texas,178,6.0
56731,2020-11-21,48.0,TX,48033.0,Borden County,134,514,"2,987",Borden,Texas,5,0.0
56732,2020-11-21,48.0,TX,48035.0,Bosque County,"4,714","13,977","50,067",Bosque,Texas,507,15.0
56733,2020-11-21,48.0,TX,48037.0,Bowie County,"23,241","71,083","257,762",Bowie,Texas,2479,97.0


In [48]:
trp_covid_cnty_cntr_merged = pd.merge(trp_covid_df_filter, tx_cnty_cntrs_df, left_on=trp_covid_df_filter["county_name"].str.lower(), right_on=tx_cnty_cntrs_df["CNAME"].str.lower(), how="inner")
trp_covid_cnty_cntr_merged
# trp_covid_geo_poly_df = pd.merge(trp_covid_df_filter, us_cnty_bndrs_df_test, on = new_cnty_clmn_nm, suffixes=("_trps_covid", "_cnty_bndrs"))

,key_0,date,State FIPS,state_postal_code,county_fips,county_name_county,pop_at_home,pop_not_at_home,trips,county_name,state,covid_cases,covid_deaths,XCoord,YCoord,CNAME
0,winkler,2020-11-02,48.0,TX,48495.0,Winkler County,"1,994","5,726","23,034",Winkler,Texas,153,5.0,-103.04717,31.849929,WINKLER
1,winkler,2020-11-03,48.0,TX,48495.0,Winkler County,"1,568","6,152","27,728",Winkler,Texas,153,5.0,-103.04717,31.849929,WINKLER
2,winkler,2020-11-04,48.0,TX,48495.0,Winkler County,"1,749","5,971","29,207",Winkler,Texas,154,5.0,-103.04717,31.849929,WINKLER
3,winkler,2020-11-05,48.0,TX,48495.0,Winkler County,"1,616","6,104","20,760",Winkler,Texas,161,5.0,-103.04717,31.849929,WINKLER
4,winkler,2020-11-06,48.0,TX,48495.0,Winkler County,"1,817","5,903","22,690",Winkler,Texas,161,5.0,-103.04717,31.849929,WINKLER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5020,loving,2020-11-17,48.0,TX,48301.0,Loving County,16,136,507,Loving,Texas,1,0.0,-103.57810,31.848756,LOVING
5021,loving,2020-11-18,48.0,TX,48301.0,Loving County,21,131,406,Loving,Texas,1,0.0,-103.57810,31.848756,LOVING
5022,loving,2020-11-19,48.0,TX,48301.0,Loving County,22,130,470,Loving,Texas,1,0.0,-103.57810,31.848756,LOVING
5023,loving,2020-11-20,48.0,TX,48301.0,Loving County,21,131,412,Loving,Texas,1,0.0,-103.57810,31.848756,LOVING


In [ ]:
# # new_cnty_fips_clmn_nm
# trp_covid_geo_poly_df = pd.merge(trp_covid_df_filter, us_cnty_bndrs_df_test, on = new_cnty_clmn_nm, suffixes=("_trps_covid", "_cnty_bndrs"))

In [ ]:
print(trp_covid_geo_poly_df.shape)
trp_covid_geo_poly_df.head()

In [51]:
trp_covid_cnty_cntr_merged = trp_covid_cnty_cntr_merged[["date", "state_postal_code", "county_name_county", "pop_at_home","pop_not_at_home", "trips", "county_name", "covid_cases", "covid_deaths", "XCoord", "YCoord"]]
trp_covid_cnty_cntr_merged.head()

,date,state_postal_code,county_name_county,pop_at_home,pop_not_at_home,trips,county_name,covid_cases,covid_deaths,XCoord,YCoord
0,2020-11-02,TX,Winkler County,"1,994","5,726","23,034",Winkler,153,5.0,-103.04717,31.849929
1,2020-11-03,TX,Winkler County,"1,568","6,152","27,728",Winkler,153,5.0,-103.04717,31.849929
2,2020-11-04,TX,Winkler County,"1,749","5,971","29,207",Winkler,154,5.0,-103.04717,31.849929
3,2020-11-05,TX,Winkler County,"1,616","6,104","20,760",Winkler,161,5.0,-103.04717,31.849929
4,2020-11-06,TX,Winkler County,"1,817","5,903","22,690",Winkler,161,5.0,-103.04717,31.849929


In [53]:
# bts_gov_trps_df_test = bts_gov_trps_df.rename(columns = {"Date": new_date_clmn_nm, "State Postal Code": new_st_pstl_code_clmn_nm, "County FIPS": new_cnty_fips_clmn_nm, "County Name": new_cnty_clmn_nm, "Population Staying at Home": new_pop_at_hm_clmn_nm, "Population Not Staying at Home": new_pop_not_at_hm_clmn_nm,"Number of Trips": new_nmbr_trps_clmn_nm})
trp_covid_cnty_cntr_merged = trp_covid_cnty_cntr_merged.rename(columns = {"XCoord": "longitude", "YCoord": "latitude"})
print(trp_covid_cnty_cntr_merged.shape)
trp_covid_cnty_cntr_merged.keys()

(5025, 11)


Index(['date', 'state_postal_code', 'county_name_county', 'pop_at_home',
       'pop_not_at_home', 'trips', 'county_name', 'covid_cases',
       'covid_deaths', 'longitude', 'latitude'],
      dtype='object')

In [ ]:
# trp_covid_geo_poly_df = trp_covid_geo_poly_df[["date", "state_postal_code_trps_covid", "county", "county_covid","pop_at_home", "pop_not_at_home", "trips", "state_trps_covid", "covid_cases", "covid_deaths", "geometry"]]
# trp_covid_geo_poly_df.head()

In [ ]:
# # bts_gov_trps_df_test = bts_gov_trps_df.rename(columns = {"Date": new_date_clmn_nm, "State Postal Code": new_st_pstl_code_clmn_nm, "County FIPS": new_cnty_fips_clmn_nm, "County Name": new_cnty_clmn_nm, "Population Staying at Home": new_pop_at_hm_clmn_nm, "Population Not Staying at Home": new_pop_not_at_hm_clmn_nm,"Number of Trips": new_nmbr_trps_clmn_nm})
# trp_covid_geo_poly_df = trp_covid_geo_poly_df.rename(columns = {"date": new_date_clmn_nm, "state_postal_code_trps_covid": new_st_pstl_code_clmn_nm, "state_trps_covid": new_st_clmn_nm})
# trp_covid_geo_poly_df.keys()

# ---------------------------------------------------------------------

#### Remove the Commas in the [clmns_to_cnvrt_null_vls_to_0] Columns Array in the State Level Dataframe

# ---------------------------------------------------------------------

In [54]:
# Remove the Commas from the [clmns_to_cnvrt_null_vls_to_0] Array.
clmns_to_cnvrt_to_int64 = ['pop_at_home', 'pop_not_at_home','trips']
trp_covid_cnty_cntr_merged_no_cmms = trp_covid_cnty_cntr_merged
trp_covid_cnty_cntr_merged_no_cmms[clmns_to_cnvrt_to_int64] = trp_covid_cnty_cntr_merged_no_cmms[clmns_to_cnvrt_to_int64].replace(',','', regex=True)
trp_covid_cnty_cntr_merged_no_cmms.head()

,date,state_postal_code,county_name_county,pop_at_home,pop_not_at_home,trips,county_name,covid_cases,covid_deaths,longitude,latitude
0,2020-11-02,TX,Winkler County,1994,5726,23034,Winkler,153,5.0,-103.04717,31.849929
1,2020-11-03,TX,Winkler County,1568,6152,27728,Winkler,153,5.0,-103.04717,31.849929
2,2020-11-04,TX,Winkler County,1749,5971,29207,Winkler,154,5.0,-103.04717,31.849929
3,2020-11-05,TX,Winkler County,1616,6104,20760,Winkler,161,5.0,-103.04717,31.849929
4,2020-11-06,TX,Winkler County,1817,5903,22690,Winkler,161,5.0,-103.04717,31.849929


In [ ]:
# # Remove the Commas from the [clmns_to_cnvrt_null_vls_to_0] Array.
# clmns_to_cnvrt_to_int64 = ['pop_at_home', 'pop_not_at_home','trips']
# trp_covid_geo_poly_df_no_cmms = trp_covid_geo_poly_df
# trp_covid_geo_poly_df_no_cmms[clmns_to_cnvrt_to_int64] = trp_covid_geo_poly_df_no_cmms[clmns_to_cnvrt_to_int64].replace(',','', regex=True)
# trp_covid_geo_poly_df_no_cmms

In [ ]:
# View Data Types
trp_covid_geo_poly_df_no_cmms.dtypes

In [56]:
# Change the Data Type of the Columns in the [clmns_to_cnvrt_null_vls_to_0] Array from a String to a int64
trp_covid_cnty_cntr_merged_no_cmms_int64 = trp_covid_cnty_cntr_merged_no_cmms
trp_covid_cnty_cntr_merged_no_cmms_int64[clmns_to_cnvrt_to_int64] = trp_covid_cnty_cntr_merged_no_cmms_int64[clmns_to_cnvrt_to_int64].astype("int64")
trp_covid_cnty_cntr_merged_no_cmms_int64.dtypes

date                  datetime64[ns]
state_postal_code             object
county_name_county            object
pop_at_home                    int64
pop_not_at_home                int64
trips                          int64
county_name                   object
covid_cases                    int64
covid_deaths                 float64
longitude                    float64
latitude                     float64
dtype: object

In [ ]:
# # Change the Data Type of the Columns in the [clmns_to_cnvrt_null_vls_to_0] Array from a String to a int64
# trp_covid_geo_poly_df_no_cmms_int64 = trp_covid_geo_poly_df_no_cmms
# trp_covid_geo_poly_df_no_cmms_int64[clmns_to_cnvrt_to_int64] = trp_covid_geo_poly_df_no_cmms[clmns_to_cnvrt_to_int64].astype("int64")

In [71]:
# Change the Data Type of the Columns in the [clmns_to_cnvrt_null_vls_to_0] Array from a String to a int64
# trp_covid_cnty_cntr_merged_no_cmms_int64 = trp_covid_geo_poly_df_no_cmms
trp_covid_cnty_cntr_merged_no_cmms_int64["covid_deaths"] = trp_covid_cnty_cntr_merged_no_cmms_int64["covid_deaths"].astype("int64")
trp_covid_cnty_cntr_merged_no_cmms_int64.dtypes

date                  datetime64[ns]
state_postal_code             object
county_name_county            object
pop_at_home                    int64
pop_not_at_home                int64
trips                          int64
county_name                   object
covid_cases                    int64
covid_deaths                   int64
longitude                    float64
latitude                     float64
dtype: object

In [ ]:
# # Change the Data Type of the Columns in the [clmns_to_cnvrt_null_vls_to_0] Array from a String to a int64
# trp_covid_geo_poly_df_no_cmms_int64 = trp_covid_geo_poly_df_no_cmms
# trp_covid_geo_poly_df_no_cmms_int64["covid_deaths"] = trp_covid_geo_poly_df_no_cmms["covid_deaths"].astype("int64")

In [72]:
# trp_covid_geo_poly_df_groupby = trp_covid_geo_poly_df
trp_covid_cnty_cntr_merged_no_cmms_int64_groupby = trp_covid_cnty_cntr_merged_no_cmms_int64.groupby("county_name_county", as_index = False).agg({"pop_at_home": "sum", "pop_not_at_home": "sum", "trips": "sum", "covid_cases": "sum", "covid_deaths": "sum", "longitude": "first", "latitude": "first"})
# trp_covid_geo_poly_df_groupby_total = trp_covid_geo_poly_df_groupby["covid_cases", "covid_deaths"].sum()
trp_covid_cnty_cntr_merged_no_cmms_int64_groupby

,county_name_county,pop_at_home,pop_not_at_home,trips,covid_cases,covid_deaths,longitude,latitude
0,Anderson County,223392,937748,3516302,60802,856,-95.652315,31.813202
1,Andrews County,91115,271445,1118513,16138,226,-102.637310,32.304938
2,Angelina County,380856,1360984,5329014,63039,1885,-94.611708,31.254821
3,Aransas County,133564,342276,1272668,7336,360,-96.998991,28.106930
4,Archer County,44140,131580,604178,3791,27,-98.687025,33.615379
...,...,...,...,...,...,...,...,...
247,Wood County,224972,677608,2353243,14926,881,-95.381974,32.786302
248,Yoakum County,43995,127825,580289,9418,184,-102.827502,33.172896
249,Young County,116510,244390,1034381,14189,285,-98.687972,33.176706
250,Zapata County,63250,220550,868278,7844,180,-99.168378,27.000585


In [ ]:
# # trp_covid_geo_poly_df_groupby = trp_covid_geo_poly_df
# trp_covid_geo_poly_df_groupby = trp_covid_geo_poly_df.groupby("county", as_index = False).agg({"pop_at_home": "sum", "pop_not_at_home": "sum", "trips": "sum", "covid_cases": "sum", "covid_deaths": "sum", "geometry": "first"})
# # trp_covid_geo_poly_df_groupby_total = trp_covid_geo_poly_df_groupby["covid_cases", "covid_deaths"].sum()
# trp_covid_geo_poly_df_groupby

In [ ]:
# Remove the Commas from the [clmns_to_cnvrt_null_vls_to_0] Array.

trp_covid_geo_poly_df_groupby_no_cmms = trp_covid_geo_poly_df_groupby
trp_covid_geo_poly_df_groupby_no_cmms["geometry"] = trp_covid_geo_poly_df_groupby_no_cmms["geometry"].replace(',','', regex=True)
trp_covid_geo_poly_df_groupby_no_cmms

In [74]:
# Export as a csv file
# File Path for Importing the bts.gov Trips Data .csv File
TEST_DATA_trp_covid_cnty_pnts_file_pth = os.path.join("Resources", "Test_Data", "TEST_DATA_trp_covid_cnty_pnts.csv")

TEST_DATA_trp_covid_cnty_pnts_df = trp_covid_cnty_cntr_merged_no_cmms_int64_groupby
TEST_DATA_trp_covid_cnty_pnts_df.to_csv(TEST_DATA_trp_covid_cnty_pnts_file_pth, index = False)

# Export to json
# trp_covid_geo_poly_df_TEST_DATA.to_json(r"Resources\Test_Data\trp_covid_geo_poly_df_TEST_DATA.json")

In [ ]:
# Export as a csv file
# File Path for Importing the bts.gov Trips Data .csv File
trp_covid_geo_poly_file_pth = os.path.join("Resources", "Test_Data", "TEST_DATA_trp_covid_geo_poly_df.csv")

TEST_DATA_trp_covid_geo_poly_df = trp_covid_geo_poly_df_groupby
TEST_DATA_trp_covid_geo_poly_df.to_csv(trp_covid_geo_poly_file_pth, index = False)

# Export to json
# trp_covid_geo_poly_df_TEST_DATA.to_json(r"Resources\Test_Data\trp_covid_geo_poly_df_TEST_DATA.json")

In [ ]:
# # Convert Dataframe to a json File
# TEST_DATA_trp_covid_geo_poly_jsonDf = TEST_DATA_trp_covid_geo_poly_df.to_json(orient='records')

# ========================================

In [ ]:
trp_covid_df_filter.head()

In [ ]:
trp_covid_df_filter.shape

In [ ]:
trp_covid_geo_poly_df.keys()

In [ ]:
trp_covid_geo_poly_df.head()

In [ ]:
trp_covid_geo_poly_df.shape

In [ ]:
trp_covid_geo_poly_df_TEST_DATA = trp_covid_geo_poly_df[["date_trps", "state_cnty_bndrs", "state_postal_code_trps_covid", "county", "pop_at_home", "pop_not_at_home", "trips", "covid_cases", "covid_deaths","geometry"]] 

In [ ]:
trp_covid_geo_poly_df_TEST_DATA.head()

In [ ]:
trp_covid_geo_poly_df_TEST_DATA = trp_covid_geo_poly_df_TEST_DATA.rename(columns = {"date_trps":"date", "state_cnty_bndrs": "state", "state_postal_code_trps_covid": "state_postal_code"})
trp_covid_geo_poly_df_TEST_DATA.keys()

In [ ]:
# Array of the Columns Where the Null Values will be Converted to 0
# These are the Columns that Should Have a Zero Values Instead of a Null Values
clmns_to_cnvrt_null_vls_to_0 =["Population Staying at Home", "Population Not Staying at Home","Number of Trips","Number of Trips <1", "Number of Trips 1-3", "Number of Trips 1-3", "Number of Trips 3-5", "Number of Trips 5-10", "Number of Trips 10-25", "Number of Trips 25-50", "Number of Trips 50-100", "Number of Trips 100-250", "Number of Trips 250-500", "Number of Trips >=500"]

In [ ]:
# Count the Null Columns
null_columns = bts_gov_trps_df.columns[bts_gov_trps_df.isnull().any()]
bts_gov_trps_df[null_columns].isnull().sum()

In [ ]:
# Array of the Columns Where the Null Values will be Converted to 0
clmns_to_dlt_st_ntnl_totls =["County FIPS", "County Name"]

In [ ]:
# Drop the Rows that Contains Null (NaN) Values in the ["County FIPS"] and ["County Name"] Columns.
bts_gov_trps_df_clnd = bts_gov_trps_df.dropna(how="all", subset=clmns_to_dlt_st_ntnl_totls)
print(bts_gov_trps_df_clnd.count())
bts_gov_trps_df_clnd

In [ ]:
bts_gov_trps_df_clnd.dtypes

In [ ]:
bts_gov_trps_df_clnd_test.dtypes

In [ ]:
bts_gov_trps_df_clnd_test_grpby = bts_gov_trps_df_clnd_test.groupby(["State Postal Code", "County Name"])
bts_gov_trps_df_clnd_test_grpby

In [ ]:
# Create Some Plots to Look at the Data

date_fltr = (bts_gov_trps_df_clnd_test['Date'] <'2020-01-01') | (bts_gov_trps_df_clnd_test['Date']>'2020-11-21')
    
#     plt.scatter(x = group[date_filter_not]['air_pressure(mBar)'],
#                 y = group[date_filter_not]["wind_speed"],

for title, group in bts_gov_trps_df_clnd_test.groupby(["County FIPS"]):
    plt.plot(group[date_fltr]['Date'], 
             group[date_fltr]['Number of Trips']) 
#              subplots=True, 
#              title=title)
    fig = plt.gcf()
    
    plt.figure(figsize=(200,100))
#     fig.savefig(f"../images/{title}_wind_speed.png")

In [ ]:
# Filter Data on ["State Postal Code"] Column for "AK"
ak_df = bts_gov_trps_df_clnd_test[bts_gov_trps_df_clnd_test["County Name"] == "Adair County"]
ak_df

In [ ]:
st_cnt = bts_gov_trps_df["Population Staying at Home"].unique()
print(st_cnt)

In [ ]:
bts_gov_trps_df["Population Staying at Home"].value_counts()

In [ ]:
bts_gov_trps_df["VA Trip Totals"] = bts_gov_trps_df["Number of Trips <1"] + bts_gov_trps_df["Number of Trips 1-3"] + bts_gov_trps_df["Number of Trips 3-5"] +  bts_gov_trps_df["Number of Trips 5-10"] + bts_gov_trps_df["Number of Trips 10-25"] + bts_gov_trps_df["Number of Trips 25-50"] + bts_gov_trps_df["Number of Trips 50-100"] +bts_gov_trps_df["Number of Trips 100-250"] +bts_gov_trps_df["Number of Trips 250-500"] +bts_gov_trps_df["Number of Trips >=500"]

bts_gov_trps_df.head()

In [ ]:
# List of all the Columns in the Trips Dataframe
bts_gov_trps_df.columns

In [ ]:
bts_gov_trps_df.describe()

In [ ]:
bts_gov_trps_df["Number of Trips"].describe()

In [ ]:
# File Path for Importing the January 21, 2020 thru November 30, 2020 COVID-19 Data .csv File
NY_Tmes_COVID_20200121_to_20201130_file_pth = os.path.join("Resources", "github.com_nytimes_covid_19_data", "us-counties20200121_to_20201130.csv")

# Convert the CSV to a Dataframe
NY_Tmes_COVID_20200121_to_20201130_df = pd.read_csv(NY_Tmes_COVID_20200121_to_20201130_file_pth)

In [ ]:
NY_Tmes_COVID_20200121_to_20201130_df.head()

In [ ]:
NY_Tmes_COVID_20200121_to_20201130_df.tail()

In [ ]:
# # File Path for Importing the January 21, 2020 thru November 30, 2020 COVID-19 Data .csv File
# NY_Tmes_COVID_20201201_file_pth = os.path.join("Resources", "github.com_nytimes_covid_19_data", "us-counties20201201.csv")

# # Convert the CSV to a Dataframe
# NY_Tmes_COVID_20201201__df = pd.read_csv(NY_Tmes_COVID_20201201_file_pth)

In [ ]:
NY_Tmes_COVID_20200121_to_20201130_df.head()

In [ ]:
# # API Keys for the Zip-Codes API
# key = "?key="
# user_API_key = "QSTIRXQR7CN0Q8T9UGRT"
# url = "https://api.zip-codes.com/ZipCodesAPI.svc/1.0/QuickGetZipCodeDetails/"

# print(url + key + user_API_key)

In [ ]:
# New Column Header Names
zip_clmn_hdr_nm = "Zip_Code"
avg_auto_insrnc_rate_clmn_hdr_nm = "Average_Auto_Insurance_Rate"

In [ ]:
# The Number of Tables on Each Website
print(len(ASN_Table_2020))
# print(len(AutoInsurance_Tables))
# print(len(State_Abbreviations_Code_Table))

# Tables from Car Insurance (www.carinsurance.com)

## The Most and Lease Expensive Auto Insurance

In [ ]:
# Table of The Most and Lease Expensive Auto Insurance, Table #5, from CarInsurance (www.carinsurance.com)
mst_and_lst_expnsv_by_state_CarInsurance_df = CarInsurance_Tables[5]
mst_and_lst_expnsv_by_state_CarInsurance_df.head()

# Resaved the DataFrame With Only the Columns Needed 
mst_and_lst_expnsv_by_state_CarInsurance_df = mst_and_lst_expnsv_by_state_CarInsurance_df[["State", "ZIP Code", "City", "Average Annual Rate"]]
mst_and_lst_expnsv_by_state_CarInsurance_df.head()

# Changed the Names of the Column Headers
mst_and_lst_expnsv_by_state_CarInsurance_df = mst_and_lst_expnsv_by_state_CarInsurance_df.rename ({"ZIP Code" : zip_clmn_hdr_nm, "Average Annual Rate" : avg_auto_insrnc_rate_clmn_hdr_nm}, axis = 1)
mst_and_lst_expnsv_by_state_CarInsurance_df.head()

## The Most Expensive Auto Insurance

In [ ]:
# Table of The Most Expensive Auto Insurance, Table #6, from CarInsurance (www.carinsurance.com)
mst_expnsv_by_state_CarInsurance_df = CarInsurance_Tables[6]
mst_expnsv_by_state_CarInsurance_df.head()

# Resaved the DataFrame With Only the Columns Needed 
mst_expnsv_by_state_CarInsurance_df = mst_expnsv_by_state_CarInsurance_df[["State", "ZIP Code", "City", "Average Annual Rate"]]
mst_expnsv_by_state_CarInsurance_df.head()

# Changed the Names of the Column Headers
mst_expnsv_by_state_CarInsurance_df = mst_expnsv_by_state_CarInsurance_df.rename ({"ZIP Code" : zip_clmn_hdr_nm, "Average Annual Rate" : avg_auto_insrnc_rate_clmn_hdr_nm}, axis = 1)
mst_expnsv_by_state_CarInsurance_df.head()

# Tables from Auto Insurance (www.autoinsurance.org)

## The 25 Most Expensive Auto Insurance

In [ ]:
# Table of The 25 Most Expensive Auto Insurance, Table #3, from AutoInsurance (www.autoinsurance.org)
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df = AutoInsurance_Tables[3]
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

# Changed the Names of the Column Headers
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df = _25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df.rename ({"ZIP Code" : zip_clmn_hdr_nm, "Average Annual Auto Insurance Rates" : avg_auto_insrnc_rate_clmn_hdr_nm}, axis = 1)
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

# Add a New Column Called State, the Column is Empty
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df["State"] = ""
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

# Reorder the Columns
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df = _25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df[["State", zip_clmn_hdr_nm, "City", avg_auto_insrnc_rate_clmn_hdr_nm]]
_25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

## The 25 Lease Expensive Auto Insurance

In [ ]:
# Table of The 25 Lowest Expensive Auto Insurance, Table #4, from AutoInsurance (www.autoinsurance.org)
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df = AutoInsurance_Tables[4]
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

# Changed the Names of the Column Headers
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df = _25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df.rename ({"ZIP Code" : zip_clmn_hdr_nm, "Average Annual Auto Insurance Rates" : avg_auto_insrnc_rate_clmn_hdr_nm}, axis = 1)
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

# Add a New Column Called State, the Column is Empty
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df["State"] = ""
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

# Reorder the Columns
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df = _25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df[["State", zip_clmn_hdr_nm, "City", avg_auto_insrnc_rate_clmn_hdr_nm]]
_25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df.head()

## The Highest and Lowest Auto Insurance Rates

In [ ]:
# Table of The Highest and Lowest Auto Insurance Rate, Table #5, from AutoInsurance (www.autoinsurance.org)
hghst_and_lwst_avg_annl_auto_insrnc_rates_per_state_df = AutoInsurance_Tables[5]
hghst_and_lwst_avg_annl_auto_insrnc_rates_per_state_df.head()

### Create a DataFrame for the Least Expensive Auto Insurance Rates from hghst_and_lwst_avg_annl_auto_insrnc_rates_per_state_df (Above)

In [ ]:
# Splitting Up The Table of The Highest and Lowest Auto Insurance Rate
lwst_avg_annl_auto_insrnc_rates_per_state_df = hghst_and_lwst_avg_annl_auto_insrnc_rates_per_state_df[["State", "Least Expensive Zip Code", "City", "Lowest Average Annual Auto Insurance Rates"]]
lwst_avg_annl_auto_insrnc_rates_per_state_df.head()

# Reorganize Column Headers and Rename Some of the Column Headers
lwst_avg_annl_auto_insrnc_rates_per_state_df = lwst_avg_annl_auto_insrnc_rates_per_state_df.rename ({"Least Expensive Zip Code" : zip_clmn_hdr_nm, "Lowest Average Annual Auto Insurance Rates" : avg_auto_insrnc_rate_clmn_hdr_nm}, axis = 1)
lwst_avg_annl_auto_insrnc_rates_per_state_df.head()

### Create a DataFrame for the Most Expensive Auto Insurance Rates from hghst_and_lwst_avg_annl_auto_insrnc_rates_per_state_df (Two Cells Above)

In [ ]:
# Splitting Up The Table of The Highest and Lowest Auto Insurance Rate
hghst_avg_annl_auto_insrnc_rates_per_state_df = hghst_and_lwst_avg_annl_auto_insrnc_rates_per_state_df[["State", "Most Expensive Zip Code", "City.1", "Highest Average Annual Auto Insurance Rates"]]
hghst_avg_annl_auto_insrnc_rates_per_state_df.head()

# Reorganize Column Headers and Rename Some of the Column Headers
hghst_avg_annl_auto_insrnc_rates_per_state_df = hghst_avg_annl_auto_insrnc_rates_per_state_df.rename ({"Most Expensive Zip Code" : zip_clmn_hdr_nm, "City.1" : "City", "Highest Average Annual Auto Insurance Rates" : avg_auto_insrnc_rate_clmn_hdr_nm}, axis = 1)
hghst_avg_annl_auto_insrnc_rates_per_state_df.head()

# Combine All of the DataFrames into One DataFrame

In [ ]:
# Create a Variable with All of the Average Auto Insurace Tables (DataFrames)
lst_of_tbls = [mst_expnsv_by_state_CarInsurance_df, _25_mst_expnsv_auto_insrnc_rts_by_zip_cde_df, _25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df, _25_lst_expnsv_auto_insrnc_rts_by_zip_cde_df, lwst_avg_annl_auto_insrnc_rates_per_state_df, hghst_avg_annl_auto_insrnc_rates_per_state_df]

# Concatenate All of the Average Auto Insurace Tables Together
avg_auto_insnc_concat_df = pd.concat(lst_of_tbls)
avg_auto_insnc_concat_df


## This Will Allow You to Reset the Index After Concatenating the DataFrames Together
# avg_auto_insnc_concat_df = pd.concat(lst_of_tbls, ignore_index = 'True')

# Data Clean UP

## Delete Dulpicate Data

### Deleted Rows with "Get Your Rates Quote Now" in Them

In [ ]:
# Evaluate Duplicate Data by Displaying the Number of Unique Values in the DataFrame
avg_auto_insnc_concat_df[zip_clmn_hdr_nm].value_counts()

In [ ]:
# Display Rows with "Get Your Rates Quote Now" in Them
avg_auto_insnc_concat_df[(avg_auto_insnc_concat_df[zip_clmn_hdr_nm] == "Get Your Rates Quote Now")]

In [ ]:
# Filtered the DataFrame then Created an Index List of all the Rows with "Get Your Rates Quote Now" in the City Column
drp_gt_your_rts_qute_nw_index = avg_auto_insnc_concat_df[(avg_auto_insnc_concat_df["City"] == "Get Your Rates Quote Now")].index
drp_gt_your_rts_qute_nw_index

In [ ]:
# Used the above Index List to drop all the rows with "Get Your Rates Quote Now"
avg_auto_insnc_concat_df = avg_auto_insnc_concat_df.drop(drp_gt_your_rts_qute_nw_index)
avg_auto_insnc_concat_df

In [ ]:
# Make Sure the Data was Deleted by Displaying the Number of Unique Values in the DataFrameNumber of Unique Values in the DataFrame
avg_auto_insnc_concat_df[zip_clmn_hdr_nm].value_counts()

### Delete the Duplicate Zip Codes from the DataFrame

#### Evaluate Duplicate Data

In [ ]:
# Evaluate Duplicate Data by Displaying the Number of Unique Values in the DataFrame Number
avg_auto_insnc_concat_df[zip_clmn_hdr_nm].value_counts()

#### Delete the Duplicate Rows in The DataFrame

In [ ]:
# Delete Rows with the Same Data
avg_auto_insnc_concat_df_drp_dplcts = avg_auto_insnc_concat_df.drop_duplicates()
avg_auto_insnc_concat_df_drp_dplcts

In [ ]:
# Make Sure the Data was Deleted by Displaying the Number of Unique Values in the DataFrameNumber of Unique Values in the DataFrame
avg_auto_insnc_concat_df_drp_dplcts[zip_clmn_hdr_nm].value_counts()

#### Reindex the DataFrame

In [ ]:
# Make an List of Values Incrementing by One
avg_auto_insnc_concat_index_list = []

for a in range(len(avg_auto_insnc_concat_df_drp_dplcts)):
#     print(a)
    avg_auto_insnc_concat_index_list.append(a)
avg_auto_insnc_concat_index_list

# Add a New Column Called Index and Append/Populate the Index Column with the Values from the avg_auto_insnc_concat_index_list List.
avg_auto_insnc_concat_df_drp_dplcts["Index"] = avg_auto_insnc_concat_index_list
avg_auto_insnc_concat_df_drp_dplcts

# Make the Index Column the Index for the DataFrame
avg_auto_insnc_concat_df_drp_dplcts = avg_auto_insnc_concat_df_drp_dplcts.set_index("Index")
avg_auto_insnc_concat_df_drp_dplcts

#### Create the .value_counts Series and Convert the Series into a DataFrame

In [ ]:
# Run .value_counts() on the avg_auto_insnc_concat_df DataFrame and Save the .value_count Series 
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_srs = avg_auto_insnc_concat_df_drp_dplcts[zip_clmn_hdr_nm].value_counts()
print(type(avg_auto_insnc_concat_df_drp_dplcts_val_cnts_srs))
print()

# Convert the .value_counts Series into a DataFrame and Save the DataFrame
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df = pd.DataFrame(avg_auto_insnc_concat_df_drp_dplcts_val_cnts_srs)
print(type(avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df))
print()
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df

#### Create a DataFrame of the Zip Codes with a Value Count Greater than One

In [ ]:
# Change the Zip Code Column Header to Value Count
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df = avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df.rename ({zip_clmn_hdr_nm : "Value Count"}, axis = 1)
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df

In [ ]:
# Create a New DataFrame that was Filtered the Value Counts Column for Values Greater than 1 
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1 = avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df[(avg_auto_insnc_concat_df_drp_dplcts_val_cnts_df["Value Count"] > 1)]
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1

In [ ]:
# Add a Column Header to the Index Column
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1.index.names = [zip_clmn_hdr_nm]
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1

# Make a List of Values Incrementing by One
dplcts_val_cnts_index_list = []

for a in range(len(avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1)):
#     print(a)
    dplcts_val_cnts_index_list.append(a)
dplcts_val_cnts_index_list

# Add a New Column Called Index and Append/Populate the Index Column with the Values from the index_list List.
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1["Index"] = dplcts_val_cnts_index_list
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1

# Make the Index Column the Index for the DataFrame
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1 = avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1.reset_index().set_index("Index")
avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1

#### Delete the Remaining Duplicate Zip Codes

In [ ]:
# Print the length of the Average Auto Insurance DataFrame and Average Auto Insurance Value Counts DataFrame
print(f"This is the length of the Average Auto Insurance DataFrame: {len(avg_auto_insnc_concat_df_drp_dplcts)}")
print()
print(f"This is the length of the Average Auto Insurance Value Counts DataFrame: {len(avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1)}")
print()

# Find And Print the Entire Row for the Remaining Duplicate Zip Codes
for y in range(len(avg_auto_insnc_concat_df_drp_dplcts)):
    
    for h in range(len(avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1)):
        
        if avg_auto_insnc_concat_df_drp_dplcts.iloc[y][zip_clmn_hdr_nm] == avg_auto_insnc_concat_df_drp_dplcts_val_cnts_grtr_thn_1.iloc[h][zip_clmn_hdr_nm]: 
            print(f"State: {avg_auto_insnc_concat_df_drp_dplcts.iloc[y]['State']}; \tCity: {avg_auto_insnc_concat_df_drp_dplcts.iloc[y]['City']}; \tZip Code: {avg_auto_insnc_concat_df_drp_dplcts.iloc[y][zip_clmn_hdr_nm]}; \tAverage Auto Insurance Rate: {avg_auto_insnc_concat_df_drp_dplcts.iloc[y][avg_auto_insrnc_rate_clmn_hdr_nm]}")

In [ ]:
# Data Types for the Columns in the avg_auto_insnc_concat_df_drp_dplcts DataFrame
avg_auto_insnc_concat_df_drp_dplcts.dtypes

In [ ]:
# Get the Index Number for the Data Filtered on: 48201, Detroit, Blank State, $30,350.09 from the avg_auto_insnc_concat_df_drp_dplcts DataFrame
drp_dplct_48201_zip_cd_index = avg_auto_insnc_concat_df_drp_dplcts[(avg_auto_insnc_concat_df_drp_dplcts[zip_clmn_hdr_nm] == "48201") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts["City"] == "Detroit") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts["State"] == "") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts[avg_auto_insrnc_rate_clmn_hdr_nm] == "$30,350.09")].index
test = drp_dplct_48201_zip_cd_index[0]
test

In [ ]:
# Get the Index Number for the Data Filtered on: 24450, Lexington, Blank State, $2,018.13 from the avg_auto_insnc_concat_df_drp_dplcts DataFrame
drp_dplct_24450_zip_cd_index = avg_auto_insnc_concat_df_drp_dplcts[(avg_auto_insnc_concat_df_drp_dplcts[zip_clmn_hdr_nm] == "24450") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts["City"] == "Lexington") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts["State"] == "") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts[avg_auto_insrnc_rate_clmn_hdr_nm] == "$2,018.13")].index
drp_dplct_24450_zip_cd_index[0]

In [ ]:
# Get the Index Number for the Data Filtered on: 06340, Groton, Delaware, $3,896.10 from the avg_auto_insnc_concat_df_drp_dplcts DataFrame
drp_dplct_06340_zip_cd_index = avg_auto_insnc_concat_df_drp_dplcts[(avg_auto_insnc_concat_df_drp_dplcts[zip_clmn_hdr_nm] == "06340") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts["City"] == "Groton") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts["State"] == "Delaware") & \
                                                                   (avg_auto_insnc_concat_df_drp_dplcts[avg_auto_insrnc_rate_clmn_hdr_nm] == "$3,896.10")].index
drp_dplct_06340_zip_cd_index[0]

##### Drop the Remaining Duplicate Data

In [ ]:
# Print the Number of Unique Items in avg_auto_insnc_concat_df_drp_dplcts
print("Values Before Dropping the Remain Duplicates")
print(avg_auto_insnc_concat_df_drp_dplcts[zip_clmn_hdr_nm].value_counts())
print()
print()

# Drop the Zip Codes from the Cell Above (Which includes: drp_dplct_48201_zip_cd_index[0], drp_dplct_24450_zip_cd_index[0], drp_dplct_06340_zip_cd_index[0])   
# avg_auto_insnc_concat_df_clnd = avg_auto_insnc_concat_df_drp_dplcts.drop([drp_dplct_48201_zip_cd_index, drp_dplct_24450_zip_cd_index, drp_dplct_06340_zip_cd_index])
avg_auto_insnc_concat_df_clnd = avg_auto_insnc_concat_df_drp_dplcts.drop([drp_dplct_48201_zip_cd_index[0], drp_dplct_24450_zip_cd_index[0], drp_dplct_06340_zip_cd_index[0]])

# Print the Number of Unique Items in avg_auto_insnc_concat_df_clnd, Do this to Make sure the Duplicate Data was Dropped
print("Values After Dropping the Remain Duplicates")
avg_auto_insnc_concat_df_clnd[zip_clmn_hdr_nm].value_counts()

# Add a State to a Blank Row in the State Column

In [ ]:
avg_auto_insnc_concat_df_clnd["City"]

In [ ]:
# Show all the Rows with no Data in the State Column
rows_wth_out_state = avg_auto_insnc_concat_df_clnd[(avg_auto_insnc_concat_df_clnd["State"] == "")]
rows_wth_out_state

## Put the State of Michigan into the Rows with Detroit as the City

In [ ]:
# Show all the Rows with Detroit
avg_auto_insnc_concat_df_clnd[(avg_auto_insnc_concat_df_clnd["City"] == "Detroit")]

In [ ]:
# Put the State of Michigan into the Rows with Detroit as the City
for x in range(len(avg_auto_insnc_concat_df_clnd)):

    if avg_auto_insnc_concat_df_clnd.iloc[x]["City"] == "Detroit" and avg_auto_insnc_concat_df_clnd.iloc[x]["State"] == "":
        
        avg_auto_insnc_concat_df_clnd.iloc[x]['State'] = "Michigan"
        
avg_auto_insnc_concat_df_clnd[(avg_auto_insnc_concat_df_clnd["City"] == "Detroit")]

## Replace the Blank State Row with the State Abbreviations

In [ ]:
# Show all the Rows with no Data in the State Column
rows_wth_out_state_df = avg_auto_insnc_concat_df_clnd[(avg_auto_insnc_concat_df_clnd["State"] == "")]
rows_wth_out_state_df

In [ ]:
# Use the Zip Code to Pull the State Abbreviation from the Zip-Codes.com API
for n in range(len(avg_auto_insnc_concat_df_clnd)):
    
    if avg_auto_insnc_concat_df_clnd.iloc[n]["State"] == "":

        zip_code = avg_auto_insnc_concat_df_clnd.iloc[n][zip_clmn_hdr_nm]

#         Make a Request to the Zip-Codes.com API for Each Row without a State
        zip_code_response = requests.get(url + zip_code + key + user_API_key).json()
        print(f"{url}{zip_code}{key}{user_API_key}")
        
#         Input the State Abbreviation into the DataFrame
        avg_auto_insnc_concat_df_clnd.iloc[n]["State"] = zip_code_response["State"]

In [ ]:
# Check to Make Sure All the Rows Have a State
avg_auto_insnc_concat_df_clnd[(avg_auto_insnc_concat_df_clnd["State"] == "")]

In [ ]:
type(zip_code_response)

In [ ]:
zip_code_response.keys()

In [ ]:
print(json.dumps(zip_code_response, indent =4, sort_keys = True))

## Replace the State Abbrevitions with the Full State Name

### View The State Abbreviations Code Table

In [ ]:
# Table of The State Abbreviations Code Table, Table #0, from app02.clerk.org
State_Abbreviations_Code_Table_df = State_Abbreviations_Code_Table[0]
print(type(State_Abbreviations_Code_Table_df))
print(len(State_Abbreviations_Code_Table_df))
print()
State_Abbreviations_Code_Table_df.head()

In [ ]:
State_Abbreviations_Code_Table_df.keys()

In [ ]:
# Replace all the Abbreviations with the Full State Name 
for t in range(len(State_Abbreviations_Code_Table_df)):

    for i in range(len(avg_auto_insnc_concat_df_clnd)):

        if State_Abbreviations_Code_Table_df.iloc[t]["Code"] == avg_auto_insnc_concat_df_clnd.iloc[i]["State"]:
            avg_auto_insnc_concat_df_clnd.iloc[i]["State"] = State_Abbreviations_Code_Table_df.iloc[t]["Description"]

In [ ]:
# Check to Make Sure All the Abbreviations are gone
avg_auto_insnc_concat_df_clnd["State"].value_counts()

In [ ]:
avg_auto_insnc_concat_df_clnd

In [ ]:
# Check to Make Sure All the Rows Have a State
avg_auto_insnc_concat_df_clnd[(avg_auto_insnc_concat_df_clnd["State"] == "")]

## Replace the $ and Commas

In [ ]:
# Remove the $ sign and commoas from each row in the Average_Auto_Insurance_Rate column
avg_auto_insnc_concat_df_clnd[avg_auto_insrnc_rate_clmn_hdr_nm] = avg_auto_insnc_concat_df_clnd[avg_auto_insrnc_rate_clmn_hdr_nm].str.replace("$", "")
avg_auto_insnc_concat_df_clnd[avg_auto_insrnc_rate_clmn_hdr_nm] = avg_auto_insnc_concat_df_clnd[avg_auto_insrnc_rate_clmn_hdr_nm].str.replace(",", "")
avg_auto_insnc_concat_df_clnd.dtypes

In [ ]:
# Check to make sure the $ and commas were removed
avg_auto_insnc_concat_df_clnd

# Export the Average Auto Insurnace DataFrame to a csv File

In [ ]:
avg_auto_insnc_concat_df_clnd.to_csv(exprt_pth, index=True, header=True)